![](img/15_dynamics.png)

##### <u>Инициализация</u>

In [39]:
from __init__ import *
o = kf.init(symbolic=True)
t, ω, μ, ρ = o.p.t, o.W_ORB, o.MU, o.RHO 

CubeSat q: quaternion(0.5, -0.5, -0.5, -0.5)
v: [0.      0.01108 0.1108 ]
Высота орбиты: 500 км
Период орбиты: 1.57 часов
Плотность атмосферы: 5.264107768251325e-13 кг/м³


##### <u>Quaternion of ORF-fixed spacecraft</u>

In [3]:
from __init__ import *
import numpy as np
from scipy.spatial.transform import Rotation as R
o = kf.init()

def show_matrices():
    kf.my_print(f"Матрица U:", bold=True)
    print(U)
    kf.my_print(f"Матрица S:", bold=True)
    print(S)
    kf.my_print(f"Матрица A:", bold=True)
    print(A)

CubeSat q: quaternion(0.5, -0.5, -0.5, -0.5)
v: [0.      0.01108 0.1108 ]


In [ ]:
#### o.c.q[0] = np.quaternion(0.5, -0.5, -0.5, -0.5)

U, S, A, _ = kf.get_matrices(o=o, t=o.p.t, obj=o.c, n=0, q=o.c.q[0])
show_matrices()

In [13]:
kf.quart2dcm(np.quaternion(*R.from_matrix(U.T).as_quat()).conj())

array([[ 0.        ,  1.        ,  0.        ],
       [ 0.89501928,  0.        ,  0.44602745],
       [ 0.44602745,  0.        , -0.89501928]])

In [8]:
o.p.t = 1000

U, S, A, _ = kf.get_matrices(o=o, t=o.p.t, obj=o.c, n=0, q=o.c.q[0])
o.c.q[0] = np.quaternion(*R.from_matrix(U.T).as_quat()).conj()
U, S, A, _ = kf.get_matrices(o=o, t=o.p.t, obj=o.c, n=0, q=o.c.q[0])
kf.my_print(f"Required quaternion:", bold=True)
print(o.c.q[0])
kf.my_print(f"Восстановленная A:", bold=True)
print(kf.quart2dcm(o.c.q[0]))
show_matrices()

Required quaternion:
quaternion(0.162003639661323, -0.688298496828581, -0.688298496828581, -0.162003639661323)
Восстановленная A:
[[ 0.          1.          0.        ]
 [ 0.89501928  0.          0.44602745]
 [ 0.44602745  0.         -0.89501928]]
Матрица U:
[[-0.89501928  0.44602745  0.        ]
 [ 0.          0.          1.        ]
 [ 0.44602745  0.89501928  0.        ]]
Матрица S:
[[ 0.44602745  0.          0.89501928]
 [-0.80105952  0.44602745  0.39920317]
 [-0.39920317 -0.89501928  0.19894048]]
Матрица A:
[[ 0.          1.          0.        ]
 [ 0.89501928  0.          0.44602745]
 [ 0.44602745  0.         -0.89501928]]


##### <u>Компоненты производной угловой скорости</u>

In [23]:
M = 3 * μ / R_0**5 * (S @ R_orb).cross(J @ S @ R_orb)
w = Matrix(kf.get_func("w", 3, numb=False, t=t))
dw = Matrix(kf.get_func("dw", 3, numb=False, t=t))

w_diff = J.inv() @ (- (w).cross(J @ w) + M)
w_diff = A.T @ J.inv() @ (- (A @ w).cross(J @ A @ w) + M)

In [ ]:
print(f"Разложение по Тейлору:")
display(w_diff[0].subs([(w[0], w[0]+dw[0])]).diff(dw[0]).expand())

##### <u>Решение уравнений Хилла-Клохесси-Уилтшира</u> (нигде не используется в проекте)

$$r(t)=Ar(0)+B\dot{r}(0)$$
$$\dot{r}(0) = B^{-1}\left( r(t) - Ar(0) \right)$$

In [7]:
from __init__ import *
o = kf.init(symbolic=True)
t, ω, μ, ρ = o.p.t, o.W_ORB, o.MU, o.RHO 

w, t, tau = var('w_0 t tau', real=True)
r0 = kf.get_vars(name='r^0', n=3, numb=False)
v0 = kf.get_vars(name='v^0', n=3, numb=False)
r = [Function('r_x'), Function('r_y'), Function('r_z')]
v = [Function('v_x'), Function('v_y'), Function('v_z')]
o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_brf = get_state_vector(func=kf.get_func, obj='d', n=1)
o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_brf = get_state_vector(func=kf.get_func, obj='c', n=1)


# a = kf.get_geopotential_acceleration(vrs=o.v, r=[r[i](t) for i in range(3)], v=[v[i](t) for i in range(3)], w=w)
a = kf.get_full_acceleration(vrs=o.v, obj=o.f, i=0, w=w, mu=μ, rho=ρ, 
                             r=Matrix([r[i](t) for i in range(3)]), 
                             v=Matrix([v[i](t) for i in range(3)]))

eqs = [Eq(r[0](t).diff(t), v[0](t)), Eq(r[1](t).diff(t), v[1](t)), Eq(r[2](t).diff(t), v[2](t)), 
       Eq(v[0](t).diff(t), a[0]), Eq(v[1](t).diff(t), a[1]), Eq(v[2](t).diff(t), a[2])]
kf.my_print(f"Уравнения ХКУ:", bold=True)
for e in eqs:
    display(e)
    
ics={r[0](0): r0[0], r[1](0): r0[1], r[2](0): r0[2], 
     v[0](0): v0[0], v[1](0): v0[1], v[2](0): v0[2]}
    
anw1 = dsolve(eqs, [r[0](t), r[1](t), r[2](t), v[0](t), v[1](t), v[2](t)], ics=ics)
for a in anw1:
    display(a)

v: [0.  0.1 0.1]
Высота орбиты: 500 км
Период орбиты: 1.57 часов
Плотность атмосферы: 5.264107768251325e-13 кг/м³
Уравнения ХКУ:


Eq(Derivative(r_x(t), t), v_x(t))

Eq(Derivative(r_y(t), t), v_y(t))

Eq(Derivative(r_z(t), t), v_z(t))

Eq(Derivative(v_x(t), t), -2*w_0*v_z(t))

...

Eq(r_y(t), r^0_y*cos(t*w_0) + v^0_y*sin(t*w_0)/w_0)

Eq(r_z(t), 4*r^0_z + 2*v^0_x/w_0 + v^0_z*sin(t*w_0)/w_0 - (3*r^0_z + 2*v^0_x/w_0)*cos(t*w_0))

Eq(v_x(t), -6*r^0_z*w_0 - 3*v^0_x - 2*v^0_z*sin(t*w_0) + (6*r^0_z*w_0 + 4*v^0_x)*cos(t*w_0))

Eq(v_y(t), -r^0_y*w_0*sin(t*w_0) + v^0_y*cos(t*w_0))

Eq(v_z(t), v^0_z*cos(t*w_0) + (3*r^0_z*w_0 + 2*v^0_x)*sin(t*w_0))

In [10]:
from __init__ import *
o = kf.init(symbolic=True)
t, ω, μ, ρ = o.p.t, o.W_ORB, o.MU, o.RHO 

w, t, tau = var('w_0 t tau', real=True, nonzero=True)
r0 = kf.get_vars(name='r^0', n=3, numb=False)
v0 = kf.get_vars(name='v^0', n=3, numb=False)
r = [Function('r_x'), Function('r_y'), Function('r_z')]
v = [Function('v_x'), Function('v_y'), Function('v_z')]
o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_brf = get_state_vector(func=kf.get_func, obj='d', n=1)
o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_brf = get_state_vector(func=kf.get_func, obj='c', n=1)


# Упрощение
r12 = Rational(1, 2)
o.f.q = [[r12, -r12, -r12, -r12]]

o.v.SOLVER = 'rk4 hkw'
o.v.DYNAMIC_MODEL['aero drag'] = True
# a = kf.get_geopotential_acceleration(vrs=o.v, r=[r[i](t) for i in range(3)], v=[v[i](t) for i in range(3)], w=w)
a = kf.get_full_acceleration(vrs=o.v, obj=o.f, i=0, w=w, mu=μ, rho=ρ, 
                             r=Matrix([r[i](t) for i in range(3)]), 
                             v=Matrix([v[i](t) for i in range(3)]))

eqs = [Eq(r[0](t).diff(t), v[0](t)), Eq(r[1](t).diff(t), v[1](t)), Eq(r[2](t).diff(t), v[2](t)), 
       Eq(v[0](t).diff(t), a[0]), Eq(v[1](t).diff(t), a[1]), Eq(v[2](t).diff(t), a[2])]
kf.my_print(f"Уравнения ХКУ:", bold=True)
for e in eqs:
    display(e)

v: [0.  0.1 0.1]
Высота орбиты: 500 км
Период орбиты: 1.57 часов
Плотность атмосферы: 5.264107768251325e-13 кг/м³
Уравнения ХКУ:


Eq(Derivative(r_x(t), t), v_x(t))

Eq(Derivative(r_y(t), t), v_y(t))

Eq(Derivative(r_z(t), t), v_z(t))

Eq(Derivative(v_x(t), t), -2*w_0*v_z(t) - v_0**2*ρ*Abs(Abs(C_d*s^d_0*s^d_1))/(2*m_d))

Eq(Derivative(v_y(t), t), -w_0**2*r_y(t))

Eq(Derivative(v_z(t), t), 3*w_0**2*r_z(t) + 2*w_0*v_x(t))

In [11]:
ics={r[0](0): r0[0], r[1](0): r0[1], r[2](0): r0[2], 
     v[0](0): v0[0], v[1](0): v0[1], v[2](0): v0[2]}
    
anw2 = dsolve(eqs, [r[0](t), r[1](t), r[2](t), v[0](t), v[1](t), v[2](t)], ics=ics)
for a in anw2:
    display(a)

Eq(r_x(t), r^0_x - t*(6*r^0_z*w_0 + 3*v^0_x) - 2*v^0_z/w_0 + (6*r^0_z + 4*v^0_x/w_0)*sin(t*w_0) + (2*v^0_z/w_0 + 2*v_0**2*ρ*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0**2))*cos(t*w_0) + 3*t**2*v_0**2*ρ*Abs(C_d*s^d_0*s^d_1)/(4*m_d) - 2*v_0**2*ρ*sin(t*w_0)**2*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0**2) - 2*v_0**2*ρ*cos(t*w_0)**2*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0**2))

Eq(r_y(t), r^0_y*cos(t*w_0) + v^0_y*sin(t*w_0)/w_0)

Eq(r_z(t), 4*r^0_z + 2*v^0_x/w_0 - (3*r^0_z + 2*v^0_x/w_0)*cos(t*w_0) + (v^0_z/w_0 + v_0**2*ρ*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0**2))*sin(t*w_0) - t*v_0**2*ρ*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0))

Eq(v_x(t), -6*r^0_z*w_0 - 3*v^0_x - (2*v^0_z + 2*v_0**2*ρ*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0))*sin(t*w_0) + (6*r^0_z*w_0 + 4*v^0_x)*cos(t*w_0) + 3*t*v_0**2*ρ*Abs(C_d*s^d_0*s^d_1)/(2*m_d))

Eq(v_y(t), -r^0_y*w_0*sin(t*w_0) + v^0_y*cos(t*w_0))

Eq(v_z(t), (v^0_z + v_0**2*ρ*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0))*cos(t*w_0) + (3*r^0_z*w_0 + 2*v^0_x)*sin(t*w_0) - v_0**2*ρ*sin(t*w_0)**2*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0) - v_0**2*ρ*cos(t*w_0)**2*Abs(C_d*s^d_0*s^d_1)/(m_d*w_0))

In [8]:
for a in anw1:
    display(a)

Eq(r_x(t), r^0_x - t*(6*r^0_z*w_0 + 3*v^0_x) + 2*v^0_z*cos(t*w_0)/w_0 - 2*v^0_z/w_0 + (6*r^0_z + 4*v^0_x/w_0)*sin(t*w_0))

Eq(r_y(t), r^0_y*cos(t*w_0) + v^0_y*sin(t*w_0)/w_0)

Eq(r_z(t), 4*r^0_z + 2*v^0_x/w_0 + v^0_z*sin(t*w_0)/w_0 - (3*r^0_z + 2*v^0_x/w_0)*cos(t*w_0))

Eq(v_x(t), -6*r^0_z*w_0 - 3*v^0_x - 2*v^0_z*sin(t*w_0) + (6*r^0_z*w_0 + 4*v^0_x)*cos(t*w_0))

Eq(v_y(t), -r^0_y*w_0*sin(t*w_0) + v^0_y*cos(t*w_0))

Eq(v_z(t), v^0_z*cos(t*w_0) + (3*r^0_z*w_0 + 2*v^0_x)*sin(t*w_0))

In [ ]:
A = Matrix([[anw[j].rhs.diff(r0[i]) for i in range(3)] for j in range(3)])
B = Matrix([[anw[j].rhs.diff(v0[i]) for i in range(3)] for j in range(3)])
C = Matrix([[anw[j+3].rhs.diff(r0[i]) for i in range(3)] for j in range(3)])
D = Matrix([[anw[j+3].rhs.diff(v0[i]) for i in range(3)] for j in range(3)])

kf.my_print(f"Матрица А:", bold=True)
display(A)
kf.my_print(f"Матрица B:", bold=True)
display(B)
kf.my_print(f"Матрица C:", bold=True)
display(C)
kf.my_print(f"Матрица D:", bold=True)
display(D)

def A(w, t):
    return kf.vec_type([[1, 0, -6*w*t + 6*kf.sin(w*t)],
                        [0, kf.cos(w*t), 0],
                        [0, 0, 4 - 3*kf.cos(w*t)]])

def B(w, t):
    return kf.vec_type([[4*kf.sin(w*t) - 3*w*t, 0, 2*kf.cos(w*t) - 2],
                        [0, kf.sin(w*t), 0],
                        [0-2*kf.cos(w*t) + 2, 0, kf.sin(w*t)]]) / w

def C(w, t):
    return kf.vec_type([[0, 0, -6*w + 6*w*kf.cos(w*t)],
                        [0, -w*kf.sin(w*t), 0],
                        [0, 0, 3*w*kf.sin(w*t)]])

def D(w, t):
    return kf.vec_type([[4*kf.cos(w*t) - 3, 0, -2*kf.sin(w*t)],
                        [0, kf.cos(w*t), 0],
                        [2*kf.sin(w*t), 0, kf.cos(w*t)]])

def get_phi(w, t, return_split: bool = False):
    if return_split:
        return A(w, t), B(w, t), C(w, t), D(w, t)
    else:
        return kf.bmat([[A(w, t), B(w, t)], [C(w, t), D(w, t)]])
    
kf.my_print(f"Проверки на вшивость:", bold=True)
r1 = A(w,t) @ r0 + B(w,t) @ v0 
v1 = C(w,t) @ r0 + D(w,t) @ v0 
for j, rv in enumerate([r1, v1]):
    tmp = rv - Matrix([anw[i + j*3].rhs for i in range(3)])
    tmp.simplify()
    display(tmp.T)

**Работающие НУ** (DScode)

In [7]:
C = [0, 100, 0, 10, 10, 0]
r0 = Matrix([-3*C[0]*w_0*t+2*C[1]*cos(w_0*t)-2*C[2]*sin(w_0*t)+C[3],
             C[4]*sin(w_0*t)+C[5]*cos(w_0*t),
             2*C[0]+C[1]*sin(w_0*t)+C[2]*cos(w_0*t)])

v0 = Matrix([-3*C[0]*w_0-2*C[1]*w_0*sin(w_0*t)-2*C[2]*w_0*cos(w_0*t),
             C[4]*w_0*cos(w_0*t)-C[5]*w_0*sin(w_0*t),
             C[1]*w_0*cos(w_0*t)-C[2]*w_0*sin(w_0*t)])

o = kf.init()
kf.my_print(f"Specific initial position:", bold=True)
display(r0.subs(t, 0).subs(w_0, o.v.W_ORB))
kf.my_print(f"Specific initial velocity:", bold=True)
display(v0.subs(t, 0).subs(w_0, o.v.W_ORB))

v: [-0.18857002 -0.0740149   0.06690491]
Specific initial position:


Matrix([
[210],
[  0],
[  0]])

Specific initial velocity:


Matrix([
[                 0],
[0.0110847443354465],
[ 0.110847443354465]])

##### <u>Свистни</u> когда доделаешь

In [3]:
kf.talk()

Мудрецу, который спрятал свое лучшее изречение, следует отсечь руки: ибо он – вор, и украл чужую мудрость
